# Advent of Code 2024, Day 9

In [ ]:
with open('input.txt', 'r') as f:
    puzzle_input = f.read().strip()

In [ ]:
import numpy as np

## [First Puzzle:](https://adventofcode.com/2024/day/9)

In [ ]:
dense_map = np.array(list(puzzle_input))

In [ ]:
tuple_map = []
file_id = 0

for i in range(len(dense_map)):
    if i % 2 == 0:
        # File blocks
        tuple_map.extend([str(file_id)] * int(dense_map[i]))
        file_id += 1
    else:
        # Free space
        tuple_map.extend(['.'] * int(dense_map[i]))

In [ ]:
tuple_map = np.array(tuple_map)

In [ ]:
free_indices = np.argwhere(tuple_map == '.').ravel()
hole_index_count = len(free_indices)

In [ ]:
while not (tuple_map[::-1][:hole_index_count] == '.').all():
    # Leftmost free space
    free_indices = np.argwhere(tuple_map == '.').ravel()
    leftmost_hole = free_indices[0]

    # Rightmost file block
    reversed_file_indices = np.argwhere(tuple_map != '.')[::-1].ravel()
    rightmost_file = reversed_file_indices[0]

    # Move the block
    tuple_map[leftmost_hole] = tuple_map[rightmost_file]
    tuple_map[rightmost_file] = '.'

In [ ]:
file_indices = np.argwhere(tuple_map != '.').ravel()
files = tuple_map[file_indices].astype(int)
np.dot(files, file_indices)

## [Second Puzzle:](https://adventofcode.com/2024/day/9/#part2)

In [87]:
with open('input.txt', 'r') as f:
    puzzle_input = f.read().strip()

In [88]:
dense_map = np.array(list(puzzle_input))

In [89]:
tuple_map = []
file_id = 0

for i in range(len(dense_map)):
    if i % 2 == 0:
        # Files
        tuple_map.append((str(file_id), int(dense_map[i]))) # Id, size tuple
        file_id += 1
    else:
        # Holes
        tuple_map.append(('.', int(dense_map[i])))

In [90]:
tuple_map = np.array(tuple_map, dtype=[('id', 'U50'), ('size', int)])

In [91]:
def merge_consecutive_holes(tuple_map):
    merged_map = []
    current_hole_size = 0
    
    for item_id, item_size in tuple_map:
        if item_id == '.':
            current_hole_size += item_size
        else:
            if current_hole_size > 0:
                merged_map.append(('.', current_hole_size))
                current_hole_size = 0
            merged_map.append((item_id, item_size))
    if current_hole_size > 0:
        merged_map.append(('.', current_hole_size))
    return np.array(merged_map, dtype=[('id', 'U50'), ('size', int)])

In [92]:
tuple_map

array([('0', 6), ('.', 3), ('1', 3), ..., ('9998', 2), ('.', 9),
       ('9999', 3)],
      shape=(19999,), dtype=[('id', '<U50'), ('size', '<i8')])

In [93]:
hole_indices = np.argwhere(tuple_map['id'] == '.').ravel()

In [94]:
reversed_file_indices = np.argwhere(tuple_map['id'] != '.')[::-1].ravel()

In [95]:
# The highest file ID is file_id - 1 (since file_id was incremented after last file)
max_file_id = file_id - 1

# Process files in descending order by their ID
for f_id in range(max_file_id, -1, -1):
    # Merge holes before attempting to move the next file, 
    # to ensure we have correct contiguous free space representation
    tuple_map = merge_consecutive_holes(tuple_map)
    
    # Find the file in the current tuple_map
    file_positions = np.argwhere(tuple_map['id'] == str(f_id)).ravel()
    if len(file_positions) == 0:
        # If this file does not exist anymore or was somehow moved incorrectly, skip it
        continue
    file_index = file_positions[0]  # The index in tuple_map describing this file
    file_size = tuple_map[file_index]['size']
    
    # Find a suitable hole to the left that can fit the entire file
    # We'll consider holes from left to right
    hole_indices = np.argwhere(tuple_map['id'] == '.').ravel()
    moved = False
    for h in hole_indices:
        # The hole must be to the left of the file
        if h < file_index:
            hole_size = tuple_map[h]['size']
            if hole_size >= file_size:
                # We can move the file here:
                file_entry = tuple_map[file_index]
                
                # Remove the file entry from its current position
                tuple_map = np.delete(tuple_map, file_index)
                
                # Adjust indices if removal is after the hole
                if file_index < h:
                    h -= 1
                
                # Adjust the hole
                if hole_size > file_size:
                    # Reduce the hole size
                    tuple_map[h] = ('.', hole_size - file_size)
                else:
                    # Hole size == file_size, remove the hole entirely
                    tuple_map = np.delete(tuple_map, h)
                    # If we removed the hole which was before file insertion point,
                    # we do not need to adjust h since we will insert at the same position
                    # now occupied by next element.

                # Insert the file at the hole's position
                tuple_map = np.insert(tuple_map, h, file_entry)
                moved = True
                break

In [99]:
tuple_map = merge_consecutive_holes(tuple_map)

In [100]:
sparse_map = np.concatenate([[item[0]] * int(item[1]) for item in tuple_map])

In [101]:
sparse_map

array(['0', '0', '0', ..., '.', '.', '.'], shape=(71209,), dtype='<U4')

In [102]:
file_indices = np.argwhere(sparse_map != '.')[::-1].ravel()
files = sparse_map[file_indices].astype(int)
np.dot(files, file_indices)

np.int64(6311911420845)